In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de estas urls:
    ! wget -O ../Data/tags.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/tags.csv?token=AA4GFHL5DGGO66Z3XHVBCCS6V4IRM
    ! wget -O ../Data/movies.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/movies.csv?token=AA4GFHNVLL665P4RLTZNUVS6V4IUK
    ! wget -O ../Data/ratings.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/ratings.csv?token=AA4GFHKROGU63T3BVY7C6W26V4IXE
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

Data wrangling es el proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

Las tareas que involucra son
* Limpieza de datos
* Eliminación de registros duplicados
* Transformación de datos
* Discretización de variables
* Detección y filtro de outliers
* Construcción de variables dummies

Pandas provee métodos para llevar a cabo estas tareas, y en esta práctica repasaremos algunos de estos métodos.

## Dataset

En esta clase usaremos un dataset con info de películas que disponibiliza datos de movielens (https://movielens.org/).

https://grouplens.org/datasets/movielens/

http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Este conjunto de datos está conformado por varios archivos:
* **movies**: idPelicula, título y género; 

donde cada registro tiene los datos de una película

* **ratings**: idUsuario, idPelicula, rating, fecha; 

donde cada registro tienen la calificación otorgada por un usuario a una película

* **tags**: idUsuario, idPelicula, tag, fecha; 

donde cada registro tienen el tag que asignó un usuario a una película

## Imports

In [ ]:
import pandas as pd
import numpy as np
import re

## Ejercicio 1 - Importar datos

Leamos los datos de movies, ratings y tags desde los archivos
* ../Data/movies.csv
* ../Data/ratings.csv
* ../Data/tags.csv

en las variables 
* data_movies
* data_ratings
* data_tags

Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna. 

Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [ ]:
data_movies = pd.read_csv("../Data/movies.csv", sep=",")
print(data_movies.dtypes)
data_movies.head(3)

In [ ]:
data_ratings = pd.read_csv("../Data/ratings.csv", sep=",")
print(data_ratings.dtypes)
data_ratings.head(3)

In [ ]:
data_tags = pd.read_csv("../Data/tags.csv", sep=",")
print(data_tags.dtypes)
data_tags.head(3)

## Ejercicio 2 - duplicated, drop_duplicates

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

**2.a** Analicemos los ratings asignados por los usuarios a las películas, y detectemos aquellos usuarios que calificaron más de una película.

**2.b** Veamos si hay usuarios que calificaron más de una vez a la misma película

**2.c** Creemos un dataframe donde cada usuario haya calificado exactamente una película

In [ ]:
# 2.a

user_many_ratings = data_ratings.duplicated(subset = ["userId"], keep = "first")
#print(user_many_ratings)
any(user_many_ratings)

In [ ]:
# 2.b

user_one_movie_many_ratings = data_ratings.duplicated(subset = ["userId", "movieId"], keep = "first")
any(user_one_movie_many_ratings)

In [ ]:
# 2.c

user_one_rating = data_ratings.drop_duplicates(subset = ["userId"], keep = "first")

print(data_ratings.shape)

print(user_one_rating.shape)


In [ ]:
len(user_one_rating.userId.unique())

## Ejercicio 3 - replace

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

El campo `genre` del DataFrame data_movies tiene una lista de géneros separadas por el caracter "|"

**3.a** Una lista de valores viejos por un valor nuevo

Vamos a buscar los valores de los registros que tengan "Fantasy" como alguno sus géneros y reemplazarlos por "Fantasy"

**3.b** Una lista de valores viejos por una lista de valores nuevos

Tomamos 10 valores del campo género al azar y construimos una lista con ellos, y otra con el último género que figura en cada elemento de la primera lista. Usamos esas dos listas para reemplazar unos valores por otros.

**3.c** Un diccionario que defina los mapeos

Con las dos listas del paso anterior, construimos un diccionario que pasamos como argumento al método replace para definir algunos reemplazos.


In [ ]:
# 3.a

fantasy_pattern = "Fantasy"
fantasy_regex = re.compile(fantasy_pattern)
fantasy_matches = data_movies.genres.apply(lambda x: x if x is np.NaN else fantasy_regex.search(x))
fantasy_mask = fantasy_matches.notnull()
fantasy_genres = fantasy_matches.loc[fantasy_mask]
#print(fantasy_genres)
fantasy_genres.iloc[0].string
genres_to_replace = [x.string for x in fantasy_genres]
print(len(genres_to_replace))
genres_to_replace_unique = list(set(genres_to_replace))
print(len(genres_to_replace_unique))

data_genres_3a = data_movies.replace(genres_to_replace_unique, "Fantasy")
data_genres_3a
#data_movies

In [ ]:
# 3.b

sample_genres = np.random.choice(data_movies.genres.unique(), 10, replace= False)
print("géneros originales: ", sample_genres)

def get_last_genre(value):
    parts = value.split("|")
    last_index = len(parts) - 1
    result = parts[last_index]
    return result

sample_genres_replacement = [get_last_genre(x) for x in sample_genres]
print("géneros de reemplazo: ", sample_genres_replacement)

data_genres_3b = data_movies.replace(sample_genres, sample_genres_replacement)
data_genres_3b

In [ ]:
# 3.c
keys = sample_genres
values = sample_genres_replacement
replacement_dict = dict(zip(keys, values))
replacement_dict

In [ ]:
data_genres_3c = data_movies.replace(replacement_dict)
data_genres_3c

## Ejercicio 4 - cut

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

El método `cut` nos permite asignar categorías a intervalos numéricos.

Creemos la columna de categorías (rating_label_cut) asociadas al campo rating de data_ratings de este modo:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4


In [ ]:
# valores de corte:
bins_rating = [0, 3, 4, 5.1]
labels_rating = ["mala", "regular", "buena"]

# Obtengo una lista de intervalos
# right=False indice que el extremo derecho del intervalo no está incluído en la categoría
categories = pd.cut(data_ratings.rating, bins_rating, labels = labels_rating, right=False)
#print(categories)
data_ratings["rating_label_cut"] = categories
data_ratings.head(10)

##  Ejercicio 5 - quantile

Calculemos los cuartilos del campo timestamp de la tabla data_ratings y asociemos las etiquetas:
* muy antigua, para el primer cuartilo
* antigua, para el segundo cuartilo
* pasada, para el tercer cuartilo
* reciente, para el cuarto cuartilo

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html

In [ ]:
q1 = data_ratings.timestamp.quantile(0.25)
q2 = data_ratings.timestamp.quantile(0.5)
q3 = data_ratings.timestamp.quantile(0.75)
q4 = data_ratings.timestamp.quantile(1)

bins_timestamp = [0, q1, q2, q3, q4]
labels_timestamp = ["muy antigua", "antigua", "pasada", "reciente"]

timestamp_categories = pd.cut(data_ratings.timestamp, bins_timestamp, labels = labels_timestamp, right=False)

data_ratings["timestamp_category"] = timestamp_categories

print(data_ratings.loc[data_ratings.timestamp_category == "reciente", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "pasada", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "antigua", ].shape)
print(data_ratings.loc[data_ratings.timestamp_category == "muy antigua", ].shape)

data_ratings.head(10)

##  Ejercicio 6 - get_dummies

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

Vamos a crear ahora variables dummy para representar el campo timestamp_category del DataFrame data_ratings, y vamos a reemplazar ese campo por las variables dummies.


In [ ]:
dummies_timestamp_category = pd.get_dummies(data_ratings.timestamp_category, prefix='timestamp_category', drop_first=True)

#print(dummies_timestamp_category)

# agregamos las columnas dummies
data_ratings_with_dummy = data_ratings.join(dummies_timestamp_category)

#data_ratings_with_dummy

# quitamos la columna timestamp_category que ahora queda representada por las dummies
result = data_ratings_with_dummy.drop(labels = ["timestamp_category"], axis="columns")
result.head(5)